## **Решение кейса №1**

### **Импорт библиотек**

In [2]:
import pandas as pd
import numpy as np

### **Загрузка данных**

In [5]:
data_url: str = "https://raw.githubusercontent.com/zalig/cu-datasets/main/golden_mine.csv"

df: pd.DataFrame = pd.read_csv(data_url)
df.head()

,Payments,Costs,Profit,Age,Gender,AgeGroup
0,9400,8100,1300,19,male,18-24
1,26600,13900,12700,29,male,25-39
2,9750,6750,3000,20,male,18-24
3,29700,11200,18500,18,female,18-24
4,49700,10450,39250,21,female,18-24
